## MIIA-4203 MODELOS AVANZADOS PARA ANÁLISIS DE DATOS II


# Introducción a Sistemas de Recomendación

## Actividad 10


### Profesor: Camilo Franco (c.franco31@uniandes.edu.co)

## Actividad en grupos
### Nombres:

**Instrucciones:** Por favor escriba los nombres de los integrantes de su grupo. Esta actividad debe ser entregada a más tardar dentro de 8 días, con la respuesta para los ejercicios y preguntas en cada numeral.

En este cuadernos estudiaremos una primera introducción a los sistemas de recomendación, examinando un modelo de recomendación en base a la popularidad, sin ningún grado de personalización. En futuras actividades exploraremos otros sistemas basados en contenido o en filtrado colaborativo, los cuales toman en cuenta las preferencias de los usuarios. 

Los sistemas de recomendación están entre las aplicaciones más populares de la ciencia de datos actualmente. El marco que ofrecen estos sistemas permite representar modelos de decisión junto con métodos estadístico/computacionales para **aprender y estimar** las preferencias de los usuarios. 


## Introducción

En esta introducción vamos a construir un modelo simple basado en popularidad. Aprenderemos nociones que nos permitirán construir modelos más complejos de recomendación. 

En este cuaderno vamos a trabajar con una base de datos de películas IMDB (https://www.imdb.com/) 

En términos generales, los sistemas de recomendación pueden clasificarse en 3 tipos:

- **Recomendadores simples:** genera recomendaciones para cada usuario según la popularidad del ítem o producto. Por ejemplo, proponer películas o un género en base a su popularidad o su crítica, donde una buena crítica aproxima una mayor probabilidad de que la película sea apreciada por una audiencia "promedio".
- **Recomendadores basados en contenido:** sugiere elementos similares en función de un elemento en particular, como por ejemplo el género, la sinopsis o el reparto (actores, etc). De esta manera, si el usuario tiene unas preferencias específicas sobre un ítem/artículo en particular, también podría tener preferencia por un ítem *similar*.
- **Recomendadores por filtrado colaborativo:** estos sistemas generan recomendaciones a partir de similitudes entre usuarios, prediciendo la calificación o preferencia de un usuario sobre un ítem, o un grupo de atributos de dicho ítem, según las calificaciones y preferencias observadas de otros usuarios. 

Por último se puede considerar también un cuarto tipo de recomendadores, que son los de tipo **Híbrido**. Estos hacen referencia al uso simultaneo de las distintas metodologias antes señaladas y de múltiples fuentes de información o tecnologías específicas, como pueden ser las redes sociales o fuentes de información geográfica, demográfica, etc.

Para profundizar en los sistemas de recomendación se pueden consultar las siguientes fuentes:
- Ch. Aggarwal. Recommender Systems. Springer, 2016.
- J. Leskovec, A. Rajaraman, J. Ullman. Mining of Massive Datasets. Cambridge University Press (http://www.mmds.org/)


## 1. Sistema de recomendación simple

Los recomendadores simples generan recomendaciones basadas en su popularidad, de acuerdo con una *preferencia promedio*. 

La pregunta es,

**¿cómo medimos la popularidad de un producto/ítem?**

Una opción es la de elegir una métrica determinada para puntuar las películas, y en base a esa métrica generar un valor de preferencia por el ítem. De esta manera es posible consturir un *orden lineal* o un *ranking estricto* con las mejores películas de la base de datos IMDB.

Pasos a seguir:
- Elegir la métrica con la que se va a calificar cada película. Este punto también se puede entender como un modelo para estimar las preferencias por una película.
- Calcular el valor de preferencia sobre cada película.
- Ordenar las películas según su valor o puntuación y obtener una lista con las mejores k peliculas.

Manos a la obra, primero carguemos el conjunto de datos como un data frame en la bilbioteca Pandas:

In [1]:
# Importamos la biblioteca Pandas
import pandas as pd

# Cargamos los datos de peliculas de la base de datos IMDB
metadata = pd.read_csv('movies_metadata.csv', low_memory=False)

print(metadata.shape)
      
list(metadata)


(45466, 24)


['adult',
 'belongs_to_collection',
 'budget',
 'genres',
 'homepage',
 'id',
 'imdb_id',
 'original_language',
 'original_title',
 'overview',
 'popularity',
 'poster_path',
 'production_companies',
 'production_countries',
 'release_date',
 'revenue',
 'runtime',
 'spoken_languages',
 'status',
 'tagline',
 'title',
 'video',
 'vote_average',
 'vote_count']

In [2]:
# Así se ven los datos
metadata.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


In [3]:
metadata[['original_title', 'popularity']][1:10]

,original_title,popularity
1,Jumanji,17.015539
2,Grumpier Old Men,11.7129
3,Waiting to Exhale,3.859495
4,Father of the Bride Part II,8.387519
5,Heat,17.924927
6,Sabrina,6.677277
7,Tom and Huck,2.561161
8,Sudden Death,5.23158
9,GoldenEye,14.686036


### Rating vs. Popularidad

Con esta información vamos a construir un sistema que recomiende películas basados directamente en su *rating*, o **voto promedio**. Sin embargo, es cierto que hay otro atributo relevante como lo es el de su **popularidad**. Por ejemplo, si un solo usuario califica con 10 una pelicula, pero 5000 usuarios le dan en promedio un 8.3, solo en base al rating la primera película sería considerada mejor que la segunda.


Bajo una perspectiva de *los grandes números*, la valoración o voto promedio de una película tiende a converger hacia un valor que refleje la calidad de la película a medida que el número de votos aumenta. Con poca información, o pocos votos, es más difícil estimar la calidad promedio de la película.  


Entonces, 

**¿qué tipo de métrica nos sirve para evaluar apropiadamente la preferencia por una película?** 

Tengamos en cuenta que en base a esta métrica se tendrán mejores o peores recomendaciones!




### Ejercicio 1.1

Examine los datos y proponga una métrica que permita estimar la preferencia promedio por una película.

In [4]:
#cuantas peliculas hay sin ingresos
len(metadata) - metadata['revenue'].count()

6

In [5]:
#Validamos los tipos de datos que tiene el df
metadata.dtypes

adult                     object
belongs_to_collection     object
budget                    object
genres                    object
homepage                  object
id                        object
imdb_id                   object
original_language         object
original_title            object
overview                  object
popularity                object
poster_path               object
production_companies      object
production_countries      object
release_date              object
revenue                  float64
runtime                  float64
spoken_languages          object
status                    object
tagline                   object
title                     object
video                     object
vote_average             float64
vote_count               float64
dtype: object

In [6]:
#La preferencia promedio debe estar basada no solo en el voto de las personas si no en sus ingresos, ya que esto da una mejor idea de que peliculas tienen mayor preferencia por el publico
metadata[['original_title', 'revenue', 'popularity', 'vote_average', 'vote_count']].sort_values(by=['revenue'], ascending=False)

,original_title,revenue,popularity,vote_average,vote_count
14551,Avatar,2.787965e+09,185.070892,7.2,12114.0
26555,Star Wars: The Force Awakens,2.068224e+09,31.626013,7.5,7993.0
1639,Titanic,1.845034e+09,26.88907,7.5,7770.0
17818,The Avengers,1.519558e+09,89.887648,7.4,12000.0
25084,Jurassic World,1.513529e+09,32.790475,6.5,8842.0
...,...,...,...,...,...
19730,"[{'iso_639_1': 'en', 'name': 'English'}]",NaN,NaN,NaN,NaN
29502,マルドゥック・スクランブル 排気,NaN,NaN,NaN,NaN
29503,"[{'iso_639_1': 'ja', 'name': '日本語'}]",NaN,NaN,NaN,NaN
35586,Avalanche Sharks,NaN,NaN,NaN,NaN


In [7]:
metadata['Avg Pref'] = metadata['revenue'] * metadata['vote_average']
metadata[['original_title', 'revenue', 'popularity', 'vote_average', 'vote_count', 'Avg Pref']].sort_values(by=['Avg Pref'], ascending=False).head(20)

#Revisando 3 posibles escalas de preferencia de peliculas, la variable popularidad infla principalmente las peliculas intantiles, ademas que no se tiene una interpretacion clara de esta popularidad. por lo tanto se deja por fuera del calculo de escala de preferencia.

,original_title,revenue,popularity,vote_average,vote_count,Avg Pref
14551,Avatar,2.787965e+09,185.070892,7.2,12114.0,2.007335e+10
26555,Star Wars: The Force Awakens,2.068224e+09,31.626013,7.5,7993.0,1.551168e+10
1639,Titanic,1.845034e+09,26.88907,7.5,7770.0,1.383776e+10
17818,The Avengers,1.519558e+09,89.887648,7.4,12000.0,1.124473e+10
28830,Furious 7,1.506249e+09,27.275687,7.3,4253.0,1.099562e+10
17437,Harry Potter and the Deathly Hallows: Part 2,1.342000e+09,24.990737,7.9,6141.0,1.060180e+10
26558,Avengers: Age of Ultron,1.405404e+09,37.37942,7.3,6908.0,1.025945e+10
25084,Jurassic World,1.513529e+09,32.790475,6.5,8842.0,9.837937e+09
22110,Frozen,1.274219e+09,24.248243,7.3,5440.0,9.301799e+09
7000,The Lord of the Rings: The Return of the King,1.118889e+09,29.324358,8.1,8226.0,9.063001e+09


### Ponderación entre rating y popularidad

Una opción puede consistir en ponderar el rating que recibe una película por su número de votos. De esta manera se pueden prevenir situaciones donde una película con un voto de 10 obtenga una mejor calificación que una película con 1000 votos con un promedio de 9.5.  

De esta manera, el voto promedio ponderado $\mu_i$, de la $i$-ésima película lo definimos como:

$$
\mu_i  = \left( \frac{v_i}{v_{max}} \right) R_i 
$$

donde $v_i$ es el número de votos para la $i$-ésima película, $v_{max}$ es el máximo número de votos que recibe la película más popular, y $R$ es el rating promedio de la pelicula.


En la base de datos tenemos los valores de $v_i$ (vote_count) y $R$ (vote_average) para cada pelicula. Además podemos agregar un filtro para que considere las películas con un minimo número de votos $v_{min}$.


Nos quedamos con las columnas necesarias para calcular el rating ponderado más el link de la pagina (en caso de que esté disponible):

In [8]:
pelisR = metadata[['original_title', 'homepage', 'vote_count', 'vote_average', 'popularity']]
pelisR.shape


(45466, 5)

In [9]:
# Calificacion de peliculas basada en la ponderacion de su rating 
def rating_ponderado(x):
    """
    Input:
    x: datos de rating y votacion de las peliculas
    m: minimo numero de votos
    Output:
    rating ponderado
    """
    m = x['vote_average'].max()
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) 

Definimos una nueva respuesta 'score' y calculamos su valor con la funcion rating_ponderado().

A continuación ordenamos las películas basados en el score y ofrecemos el top 15 de películas preferidas:

In [10]:
import warnings
warnings.filterwarnings("ignore", category=Warning)

# si tiene al menos vmin votos, incluimos la pelicula
vmin = 1
pelisRF = pelisR.copy().loc[pelisR['vote_count'] >= vmin]
pelisRF['score'] = rating_ponderado(pelisRF)

# ordenamos las peliculas de acuerdo con el score
pelisRF = pelisRF.sort_values('score', ascending=False)

# Mostramos el top15 de las peliculas
pelisRF[['original_title', 'homepage', 'vote_count', 'vote_average', 'popularity', 'score']].head(15)

,original_title,homepage,vote_count,vote_average,popularity,score
10309,Dilwale Dulhania Le Jayenge,NaN,661.0,9.1,34.457024,8.964382
314,The Shawshank Redemption,NaN,8358.0,8.5,51.645403,8.489842
834,The Godfather,http://www.thegodfather.com/,6024.0,8.5,41.109264,8.485913
40251,君の名は。,https://www.funimationfilms.com/movie/yourname/,1030.0,8.5,34.461252,8.418269
39085,Planet Earth,NaN,176.0,8.8,4.501137,8.326882
12481,The Dark Knight,http://thedarkknight.warnerbros.com/dvdsite/,12269.0,8.3,123.167259,8.293240
2843,Fight Club,http://www.foxmovies.com/movies/fight-club,9678.0,8.3,63.869599,8.291433
292,Pulp Fiction,NaN,8670.0,8.3,140.950236,8.290438
522,Schindler's List,http://www.schindlerslist.com/,4436.0,8.3,41.725123,8.281332
23673,Whiplash,http://sonyclassics.com/whiplash/,4376.0,8.3,64.29999,8.281076


Examinemos el efecto de cambiar el mínimo numero de votos a considerar:

In [11]:
# si tiene al menos mil votos incluimos la pelicula
vmin = 1000
pelisRF = pelisR.copy().loc[pelisR['vote_count'] >= vmin]
pelisRF['score'] = rating_ponderado(pelisRF)

# ordenamos las peliculas de acuerdo con el score
pelisRF = pelisRF.sort_values('score', ascending=False)

# Mostramos el top15 de las peliculas
pelisRF[['original_title', 'homepage', 'vote_count', 'vote_average', 'popularity', 'score']].head(15)

,original_title,homepage,vote_count,vote_average,popularity,score
314,The Shawshank Redemption,NaN,8358.0,8.5,51.645403,8.491364
834,The Godfather,http://www.thegodfather.com/,6024.0,8.5,41.109264,8.488023
40251,君の名は。,https://www.funimationfilms.com/movie/yourname/,1030.0,8.5,34.461252,8.430429
12481,The Dark Knight,http://thedarkknight.warnerbros.com/dvdsite/,12269.0,8.3,123.167259,8.294254
2843,Fight Club,http://www.foxmovies.com/movies/fight-club,9678.0,8.3,63.869599,8.292717
292,Pulp Fiction,NaN,8670.0,8.3,140.950236,8.291871
522,Schindler's List,http://www.schindlerslist.com/,4436.0,8.3,41.725123,8.284126
23673,Whiplash,http://sonyclassics.com/whiplash/,4376.0,8.3,64.29999,8.283909
5481,千と千尋の神隠し,http://movies.disney.com/spirited-away,3968.0,8.3,41.048867,8.282258
2211,La vita è bella,NaN,3643.0,8.3,39.39497,8.280679


Veamos en qué consiste esa pelicula en japonés:

In [12]:
import webbrowser

webbrowser.open(pelisRF['homepage'][40251]) 

True

### Ejercicio 1.2

Examine cómo varían los resultados si tomamos directamente la variable de popularidad.

In [13]:
pelisRF['popularity'] = pelisRF['popularity'].astype(float)
pelisRF.sort_values(by=['popularity'], ascending=False).head(20)

#Como habiamos visto anteriormente, la popularidad impulsa en el top 5 las peliculas infantiles principalmente. 

,original_title,homepage,vote_count,vote_average,popularity,score
30700,Minions,http://www.minionsmovie.com/,4729.0,6.4,547.488298,6.388517
33356,Wonder Woman,http://www.warnerbros.com/wonder-woman,5025.0,7.2,294.337037,7.187841
42222,Beauty and the Beast,http://movies.disney.com/beauty-and-the-beast-...,5530.0,6.8,287.253654,6.789564
43644,Baby Driver,NaN,2083.0,7.2,228.032744,7.170739
24455,Big Hero 6,http://movies.disney.com/big-hero-6,6289.0,7.8,213.849907,7.789472
26564,Deadpool,http://www.foxmovies.com/movies/deadpool,11444.0,7.4,187.860492,7.394508
26566,Guardians of the Galaxy Vol. 2,http://marvel.com/movies/movie/221/guardians_o...,4858.0,7.6,185.330992,7.586726
14551,Avatar,http://www.avatarmovie.com/,12114.0,7.2,185.070892,7.194952
24351,John Wick,http://www.johnwickthemovie.com/,5499.0,7.0,183.870374,6.989197
23675,Gone Girl,http://www.gonegirlmovie.com/,6023.0,7.9,154.801009,7.888867


### Recomendación de peliculas mas populares por genero

In [14]:
import numpy as np
from ast import literal_eval

# trabajamos la informacion por generos
metadata['genres'] = metadata['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

# añadimos la variable del año
metadata['year'] = pd.to_datetime(metadata['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

metadata.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,Avg Pref,year
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,2.876366e+09,1995
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1.813301e+09,1995
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,0.000000e+00,1995


Primero nos quedamos con todos los generos:

In [15]:
generos = metadata.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
generos.name = 'genre'
gen_md = metadata.drop('genres', axis=1).join(generos)

gen_md.head(6)

,adult,belongs_to_collection,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,...,spoken_languages,status,tagline,title,video,vote_average,vote_count,Avg Pref,year,genre
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,2.876366e+09,1995,Animation
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,2.876366e+09,1995,Comedy
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,2.876366e+09,1995,Family
1,False,NaN,65000000,NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1.813301e+09,1995,Adventure
1,False,NaN,65000000,NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1.813301e+09,1995,Fantasy
1,False,NaN,65000000,NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1.813301e+09,1995,Family


Construimos una funcion para un género particular y que tome en cuenta peliculas con un número vmin de votos:

In [16]:
def rec_gen(genero, vmin):
    df = gen_md[gen_md['genre'] == genero]
    v = df[df['vote_count'].notnull()]['vote_count'].astype('int')
    R = df[df['vote_average'].notnull()]['vote_average'].astype('int')
    m = df['vote_average'].max()
    
    pelisG = df[(df['vote_count'] >= vmin) & (df['vote_count'].notnull()) & (df['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity', 'overview', 'homepage']]
    pelisG['vote_count'] = pelisG['vote_count'].astype('int')
    pelisG['vote_average'] = pelisG['vote_average'].astype('int')
    
    pelisG['wr'] = v/m * R
    pelisG = pelisG.sort_values('wr', ascending=False).head(250)
    
    return pelisG

Veamos el Top-15 de recomendaciones en Ciencia Ficción:

In [17]:
k = 15
scifi = rec_gen('Science Fiction', 1000)
scifi.head(15)

,title,year,vote_count,vote_average,popularity,overview,homepage,wr
15480,Inception,2010,14075,8,29.108149,"Cobb, a skilled thief who commits corporate es...",http://inceptionmovie.warnerbros.com/,11260.0
22879,Interstellar,2014,11187,8,32.213481,Interstellar chronicles the adventures of a gr...,http://www.interstellarmovie.net/,8949.6
14551,Avatar,2009,12114,7,185.070892,"In the 22nd century, a paraplegic Marine is di...",http://www.avatarmovie.com/,8479.8
17818,The Avengers,2012,12000,7,89.887648,When an unexpected enemy emerges and threatens...,http://marvel.com/avengers_movie/,8400.0
23753,Guardians of the Galaxy,2014,10014,7,53.291601,"Light years from Earth, 26 years after being a...",http://marvel.com/guardians,7009.8
26553,Mad Max: Fury Road,2015,9629,7,29.36178,An apocalyptic story set in the furthest reach...,http://www.madmaxmovie.com/,6740.3
2458,The Matrix,1999,9079,7,33.366332,"Set in the 22nd century, The Matrix tells the ...",http://www.warnerbros.com/matrix,6355.3
12588,Iron Man,2008,8951,7,22.073099,"After being held captive in an Afghan cave, bi...",http://www.ironmanmovie.com/,6265.7
18244,The Hunger Games,2012,9634,6,20.031667,Every year in the ruins of what was once North...,http://www.thehungergames.movie/,5780.4
26555,Star Wars: The Force Awakens,2015,7993,7,31.626013,Thirty years after defeating the Galactic Empi...,http://www.starwars.com/films/star-wars-episod...,5595.1


Inception e Interstellar aparecen en las dos primeras posiciones. Personalmente me gusta más Interstellar (si quieres ver un agujero negro, esta película es lo mejor que podrás conseguir), pero reconozco que Inception tiene mucho nivel. Podemos seguir refinando este tipo de recomendaciones prestando atención a los distintos atributos que tenemos disponibles sobre las películas. 


### Ejercicio 1.3

Elija el género de su interés y aplique la métrica propuesta en el Ejercicio 1.1

In [26]:
def rec_gen_custom(genero, vmin):
    df = gen_md[gen_md['genre'] == genero]
    
    pelisG = df[(df['vote_count'] >= vmin) & (df['vote_count'].notnull()) & (df['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity', 'overview', 'homepage','revenue']]
    pelisG['vote_count'] = pelisG['vote_count'].astype('int')
    pelisG['vote_average'] = pelisG['vote_average'].astype('int')
    
    pelisG['my_metric'] = pelisG['revenue']*pelisG['vote_average']
    pelisG = pelisG.sort_values('my_metric', ascending=False).head(250)
    
    return pelisG

In [41]:
scifi = rec_gen_custom('Animation', 1000)
scifi.head(15)

,title,year,vote_count,vote_average,popularity,overview,homepage,revenue,my_metric
22110,Frozen,2013,5440,7,24.248243,Young princess Anna of Arendelle dreams about ...,http://movies.disney.com/frozen,1.274219e+09,8.919533e+09
15348,Toy Story 3,2010,4710,7,16.96647,"Woody, Buzz, and the rest of Andy's toys haven...",http://disney.go.com/toystory/,1.066970e+09,7.468788e+09
36253,Zootopia,2016,4961,7,26.024868,"Determined to prove herself, Officer Judy Hopp...",http://movies.disney.com/zootopia,1.023784e+09,7.166489e+09
30700,Minions,2015,4729,6,547.488298,"Minions Stuart, Kevin and Bob are recruited by...",http://www.minionsmovie.com/,1.156731e+09,6.940386e+09
21161,Despicable Me 2,2013,4729,7,24.82355,Gru is recruited by the Anti-Villain League to...,http://www.despicableme.com/,9.707619e+08,6.795333e+09
6232,Finding Nemo,2003,6292,7,25.497794,"Nemo, an adventurous young clownfish, is unexp...",http://movies.disney.com/finding-nemo,9.403355e+08,6.582349e+09
359,The Lion King,1994,5520,8,21.605761,A young lion cub named Simba can't wait to be ...,http://movies.disney.com/the-lion-king,7.882418e+08,6.305934e+09
38176,Finding Dory,2016,4333,6,14.477677,Dory is reunited with her friends Nemo and Mar...,http://movies.disney.com/finding-dory,1.028571e+09,6.171425e+09
44009,Despicable Me 3,2017,2002,6,36.631519,Gru and his wife Lucy must stop former '80s ch...,http://www.despicable.me,1.020063e+09,6.120380e+09
30315,Inside Out,2015,6737,7,23.985587,"Growing up can be a bumpy road, and it's no ex...",http://movies.disney.com/inside-out,8.576112e+08,6.003278e+09


Nuestro género favorito (decido por votación) es animación donde efectivamente, revisando el top 3, son películas de buena calificación y taquilleras, esto hace que tengan un alta probabilidad de ser vistas si se elige este genero. A Juan Camilo le gusta Toy Story 3 por encima de las otras Toy Story por haber crecido junto a ellas, a Camilo Zootopia y Minions y a Carlos Frozen porque su sobrina cantaba Libre Soy todo el tiempo, por ende, vemos que la métrica propuesta con estas 2 variables (vote_average y revenue) es acertada.